In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data.csv")

In [2]:
patient_id = list(df["Patient ID"])
recording_loc = list(df["Recording locations:"])
murmur = list(df["Murmur"])
murmur_loc = list(df["Murmur locations"])

In [3]:
recording_loc = [x.split("+") for x in recording_loc]

for i in range(len(murmur_loc)):
    if murmur_loc[i] is np.nan:
        murmur_loc[i] = []
    else:
        murmur_loc[i] = murmur_loc[i].split("+")

In [4]:
X = []
y = []

for i in range(len(patient_id)):
    if murmur[i] == "Absent":
        for recloc in recording_loc[i]:
            X.append(np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')[:200])
            y.append([1, 0, 0, 0, 0])
    elif murmur[i] == "Present":
        for recloc in recording_loc[i]:
            if recloc in murmur_loc[i]:
                if recloc == "AV":
                    X.append(np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')[:200])
                    y.append([0, 1, 0, 0, 0])
                elif recloc == "PV":
                    X.append(np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')[:200])
                    y.append([0, 0, 1, 0, 0])
                elif recloc == "TV":
                    X.append(np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')[:200])
                    y.append([0, 0, 0, 1, 0])
                elif recloc == "MV":
                    X.append(np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')[:200])
                    y.append([0, 0, 0, 0, 1])
                else:
                    print("UNUSUAL RECLOC:", recloc)
                    print("NOTHING APPENDED.... CONTINUING")
            else:
                X.append(np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')[:200])
                y.append([1, 0, 0, 0, 0])
    #print("Completed patient #", str(i+1))

UNUSUAL RECLOC: Phc
NOTHING APPENDED.... CONTINUING
UNUSUAL RECLOC: Phc
NOTHING APPENDED.... CONTINUING


In [5]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.2)

In [6]:
currents = []
indices = []
for i in range(len(X_train)):
    if y_train[i] != [1, 0, 0, 0, 0]:
        current = np.array(X_train[i]).transpose()
        indices.append(i)
        currents.append(current)
    
len(currents)

388

In [7]:
from scipy.stats import skew, kurtosis

moments = []

for sample in currents:
    output = []
    for row in sample:
        output.append(np.mean(row))
        output.append(np.var(row))
        output.append(skew(row))
        output.append(kurtosis(row))
    moments.append(output)
    
moments = np.array(moments)
print(moments)
print(moments.shape)

[[4.53442701e+05 8.89338860e+11 3.23523500e+00 ... 6.90913922e+08
  5.50157885e+00 3.29543208e+01]
 [6.91447470e+04 3.46480597e+10 5.48939222e+00 ... 1.13169499e+13
  5.15638992e+00 2.66285171e+01]
 [2.17128432e+04 2.23557442e+10 1.25690684e+01 ... 4.85184166e+08
  7.95074869e+00 8.11676686e+01]
 ...
 [1.46710531e+05 1.08316095e+11 4.28363028e+00 ... 1.21121195e+12
  9.45193546e+00 1.01410083e+02]
 [1.21084955e+04 1.98201078e+10 1.39959802e+01 ... 9.97853221e+12
  1.40358244e+01 1.95004589e+02]
 [5.95486322e+04 9.44801762e+09 3.45404886e+00 ... 4.40007675e+09
  5.45879660e+00 3.07800976e+01]]
(388, 72)


In [8]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
y_train = np.array(y_train)
moments_res, y_res = sm.fit_resample(moments, y_train[indices])

In [9]:
for i in range(len(moments_res)):
    if moments_res[i] not in moments:
        print(i)

388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423


In [21]:
from __future__ import print_function, division

#from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers import LeakyReLU
from keras.layers import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers.legacy import Adam
from keras import losses
from tensorflow.keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

import sounddevice as sd 
from python_speech_features import mfcc
import python_speech_features as psf
import scipy.io.wavfile as wav
import wavio as wv
import xlwt
from xlwt import Workbook

In [22]:
class BIGAN():
    def __init__(self):
        self.img_rows = 200 # 28
        self.img_cols = 18 # 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 72

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)
        
    def build_encoder(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.latent_dim))

        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)
    
    def build_generator(self):
        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim, ))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(1024)(d_in)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)
    
    def train(self, epochs, batch_size=10, sample_interval=50):

        # Load the dataset
        dataset = np.array(X_train)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        d_losses = []
        d_accuracies = []
        g_losses = []
        g_accuracies = []

        for epoch in range(epochs):


            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Sample noise and generate img
            z = np.random.normal(size=(batch_size, self.latent_dim))
            imgs_ = self.generator.predict(z)

            # Select a random batch of images and encode
            idx = np.random.randint(0, dataset.shape[0], batch_size)
            imgs = dataset[idx]
            z_ = self.encoder.predict(imgs)

            # Train the discriminator (img -> z is valid, z -> img is fake)
            d_loss_real = self.discriminator.train_on_batch([z_, imgs], valid)
            d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            d_losses.append(d_loss[0])
            d_accuracies.append(d_loss[1])

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (z -> img is valid and img -> z is is invalid)
            g_loss = self.bigan_generator.train_on_batch([z, imgs], [valid, fake])
            g_losses.append(g_loss[0])
            g_accuracies.append(g_loss[1])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))


            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_interval(epoch)

        plt.plot(list(range(1, epochs+1)), d_losses, label = "discriminator loss")
        plt.title('model loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.legend()
        plt.savefig("./GAN Plots/d_losses.png")
        plt.close()
        plt.plot(list(range(1, epochs+1)), g_losses, label = "generator loss")
        plt.title('model loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.legend()
        plt.savefig("./GAN Plots/g_losses.png")
        plt.close()
        plt.plot(list(range(1, epochs+1)), d_accuracies, label = "discriminator accuracy")
        plt.title('model accuracy')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.legend()
        plt.savefig("./GAN Plots/d_accuracies.png")
        plt.close()
        plt.plot(list(range(1, epochs+1)), g_accuracies, label = "generator accuracy")
        plt.title('model accuracy')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.legend()
        plt.savefig("./GAN Plots/g_accuracies.png")
        plt.close()

    def sample_interval(self, epoch):
        r, c = 5, 5
        z = np.random.normal(size=(25, self.latent_dim))
        gen_imgs = self.generator.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("./GAN Plots/gan_%d.png" % epoch)
        plt.close()

In [23]:
bigan = BIGAN()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               37376     
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_8 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dense_25 (Dense)            (None, 512)               262656    
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_9 (Bat  (None, 512)               2048      
 chNormalization)                                     

In [25]:
bigan.train(epochs=1000)

1/1 [==============================] - 0s 16ms/step
0 [D loss: 76755.012170, acc: 55.00%] [G loss: 42267528.000000]
1/1 [==============================] - 0s 14ms/step
1 [D loss: 34992.354418, acc: 60.00%] [G loss: 67500208.000000]
1/1 [==============================] - 0s 14ms/step
2 [D loss: 3894896.388616, acc: 50.00%] [G loss: 180254112.000000]
1/1 [==============================] - 0s 28ms/step
3 [D loss: 89202.775885, acc: 55.00%] [G loss: 53084584.000000]
1/1 [==============================] - 0s 25ms/step
4 [D loss: 0.482968, acc: 55.00%] [G loss: 37430288.000000]
1/1 [==============================] - 0s 20ms/step
5 [D loss: 0.535867, acc: 55.00%] [G loss: 64591476.000000]
1/1 [==============================] - 0s 16ms/step
6 [D loss: 811233.266011, acc: 65.00%] [G loss: 101532512.000000]
1/1 [==============================] - 0s 14ms/step
7 [D loss: 0.536508, acc: 60.00%] [G loss: 21626374.000000]
1/1 [==============================] - 0s 25ms/step
8 [D loss: 0.575800, acc: 5

1/1 [==============================] - 0s 14ms/step
49 [D loss: 0.672917, acc: 55.00%] [G loss: 190414944.000000]
1/1 [==============================] - 0s 17ms/step
50 [D loss: 0.991697, acc: 55.00%] [G loss: 247578016.000000]
1/1 [==============================] - 0s 12ms/step
51 [D loss: 0.992629, acc: 50.00%] [G loss: 64751668.000000]
1/1 [==============================] - 0s 19ms/step
52 [D loss: 0.812733, acc: 55.00%] [G loss: 438714720.000000]
1/1 [==============================] - 0s 4ms/step
53 [D loss: 0.974056, acc: 65.00%] [G loss: 274009760.000000]
1/1 [==============================] - 0s 16ms/step
54 [D loss: 1.122019, acc: 55.00%] [G loss: 249324448.000000]
1/1 [==============================] - 0s 16ms/step
55 [D loss: 0.947763, acc: 55.00%] [G loss: 313663424.000000]
1/1 [==============================] - 0s 808us/step
56 [D loss: 0.827212, acc: 55.00%] [G loss: 215829408.000000]
1/1 [==============================] - 0s 15ms/step
57 [D loss: 0.839778, acc: 55.00%] [G

1/1 [==============================] - 0s 17ms/step
98 [D loss: 1.540910, acc: 50.00%] [G loss: 111693392.000000]
1/1 [==============================] - 0s 20ms/step
99 [D loss: 1.117639, acc: 60.00%] [G loss: 63133184.000000]
1/1 [==============================] - 0s 14ms/step
100 [D loss: 1.622158, acc: 50.00%] [G loss: 240049968.000000]
1/1 [==============================] - 0s 17ms/step
101 [D loss: 1.500769, acc: 50.00%] [G loss: 418541344.000000]
1/1 [==============================] - 0s 17ms/step
102 [D loss: 2.155401, acc: 50.00%] [G loss: 121382640.000000]
1/1 [==============================] - 0s 15ms/step
103 [D loss: 1.040730, acc: 55.00%] [G loss: 425269952.000000]
1/1 [==============================] - 0s 17ms/step
104 [D loss: 1.469853, acc: 55.00%] [G loss: 209164768.000000]
1/1 [==============================] - 0s 16ms/step
105 [D loss: 2.062857, acc: 50.00%] [G loss: 266979632.000000]
1/1 [==============================] - 0s 16ms/step
106 [D loss: 1.464725, acc: 50.

1/1 [==============================] - 0s 19ms/step
147 [D loss: 1.599668, acc: 55.00%] [G loss: 510195136.000000]
1/1 [==============================] - 0s 16ms/step
148 [D loss: 1.972119, acc: 60.00%] [G loss: 462088384.000000]
1/1 [==============================] - 0s 16ms/step
149 [D loss: 2.151421, acc: 55.00%] [G loss: 119297896.000000]
1/1 [==============================] - 0s 18ms/step
150 [D loss: 0.691055, acc: 65.00%] [G loss: 509910432.000000]
1/1 [==============================] - 0s 21ms/step
151 [D loss: 1.974315, acc: 55.00%] [G loss: 861718528.000000]
1/1 [==============================] - 0s 16ms/step
152 [D loss: 1.948912, acc: 50.00%] [G loss: 406581600.000000]
1/1 [==============================] - 0s 16ms/step
153 [D loss: 1.976827, acc: 55.00%] [G loss: 400364736.000000]
1/1 [==============================] - 0s 16ms/step
154 [D loss: 241200.353324, acc: 45.00%] [G loss: 609200640.000000]
1/1 [==============================] - 0s 21ms/step
155 [D loss: 2.943336, 

1/1 [==============================] - 0s 16ms/step
196 [D loss: 2.761037, acc: 50.00%] [G loss: 322371104.000000]
1/1 [==============================] - 0s 15ms/step
197 [D loss: 2.105011, acc: 55.00%] [G loss: 208060480.000000]
1/1 [==============================] - 0s 17ms/step
198 [D loss: 2.175211, acc: 55.00%] [G loss: 1555563648.000000]
1/1 [==============================] - 0s 17ms/step
199 [D loss: 2.280545, acc: 55.00%] [G loss: 440550464.000000]
1/1 [==============================] - 0s 17ms/step
200 [D loss: 3.195590, acc: 50.00%] [G loss: 613434752.000000]
1/1 [==============================] - 0s 24ms/step
201 [D loss: 2.262653, acc: 60.00%] [G loss: 501425728.000000]
1/1 [==============================] - 0s 16ms/step
202 [D loss: 2.792910, acc: 55.00%] [G loss: 481241664.000000]
1/1 [==============================] - 0s 15ms/step
203 [D loss: 2.615844, acc: 50.00%] [G loss: 554356352.000000]
1/1 [==============================] - 0s 14ms/step
204 [D loss: 2.810334, acc:

...

1/1 [==============================] - 0s 17ms/step
780 [D loss: 8.425238, acc: 50.00%] [G loss: 504100864.000000]
1/1 [==============================] - 0s 16ms/step
781 [D loss: 10.481197, acc: 50.00%] [G loss: 1102978176.000000]
1/1 [==============================] - 0s 18ms/step
782 [D loss: 7.634022, acc: 50.00%] [G loss: 507382464.000000]
1/1 [==============================] - 0s 17ms/step
783 [D loss: 12.074661, acc: 50.00%] [G loss: 3322475776.000000]
1/1 [==============================] - 0s 16ms/step
784 [D loss: 7.820397, acc: 55.00%] [G loss: 1003980288.000000]
1/1 [==============================] - 0s 19ms/step
785 [D loss: 8.706353, acc: 50.00%] [G loss: 969872768.000000]
1/1 [==============================] - 0s 16ms/step
786 [D loss: 11.021360, acc: 50.00%] [G loss: 582807680.000000]
1/1 [==============================] - 0s 19ms/step
787 [D loss: 8.642257, acc: 50.00%] [G loss: 3831348736.000000]
1/1 [==============================] - 0s 19ms/step
788 [D loss: 7.611562

1/1 [==============================] - 0s 16ms/step
829 [D loss: 10.138712, acc: 50.00%] [G loss: 2639365888.000000]
1/1 [==============================] - 0s 19ms/step
830 [D loss: 10.274395, acc: 50.00%] [G loss: 1347077120.000000]
1/1 [==============================] - 0s 16ms/step
831 [D loss: 6.979717, acc: 50.00%] [G loss: 531568544.000000]
1/1 [==============================] - 0s 16ms/step
832 [D loss: 7.308295, acc: 50.00%] [G loss: 1374061568.000000]
1/1 [==============================] - 0s 17ms/step
833 [D loss: 8.859622, acc: 50.00%] [G loss: 376832192.000000]
1/1 [==============================] - 0s 17ms/step
834 [D loss: 8.635649, acc: 50.00%] [G loss: 3119064576.000000]
1/1 [==============================] - 0s 16ms/step
835 [D loss: 7.383510, acc: 50.00%] [G loss: 813955648.000000]
1/1 [==============================] - 0s 15ms/step
836 [D loss: 9.288500, acc: 50.00%] [G loss: 2590238720.000000]
1/1 [==============================] - 0s 18ms/step
837 [D loss: 8.429510

1/1 [==============================] - 0s 18ms/step
877 [D loss: 9.370600, acc: 50.00%] [G loss: 2231959040.000000]
1/1 [==============================] - 0s 15ms/step
878 [D loss: 6.246524, acc: 55.00%] [G loss: 905433728.000000]
1/1 [==============================] - 0s 18ms/step
879 [D loss: 8.768115, acc: 55.00%] [G loss: 465344928.000000]
1/1 [==============================] - 0s 17ms/step
880 [D loss: 7.116652, acc: 55.00%] [G loss: 804431104.000000]
1/1 [==============================] - 0s 18ms/step
881 [D loss: 6.504863, acc: 50.00%] [G loss: 13364791296.000000]
1/1 [==============================] - 0s 17ms/step
882 [D loss: 10.176625, acc: 50.00%] [G loss: 1306474496.000000]
1/1 [==============================] - 0s 20ms/step
883 [D loss: 12.019208, acc: 50.00%] [G loss: 1272112512.000000]
1/1 [==============================] - 0s 16ms/step
884 [D loss: 10.823301, acc: 50.00%] [G loss: 4456844288.000000]
1/1 [==============================] - 0s 17ms/step
885 [D loss: 10.029

1/1 [==============================] - 0s 17ms/step
926 [D loss: 8.013535, acc: 50.00%] [G loss: 7807912448.000000]
1/1 [==============================] - 0s 18ms/step
927 [D loss: 8.647951, acc: 50.00%] [G loss: 1698152064.000000]
1/1 [==============================] - 0s 16ms/step
928 [D loss: 8.559484, acc: 50.00%] [G loss: 693735360.000000]
1/1 [==============================] - 0s 17ms/step
929 [D loss: 6.015058, acc: 55.00%] [G loss: 1825024384.000000]
1/1 [==============================] - 0s 17ms/step
930 [D loss: 8.831512, acc: 50.00%] [G loss: 884741760.000000]
1/1 [==============================] - 0s 17ms/step
931 [D loss: 10.487745, acc: 50.00%] [G loss: 1335520128.000000]
1/1 [==============================] - 0s 17ms/step
932 [D loss: 9.694426, acc: 50.00%] [G loss: 1284727040.000000]
1/1 [==============================] - 0s 17ms/step
933 [D loss: 11.270846, acc: 50.00%] [G loss: 2809579776.000000]
1/1 [==============================] - 0s 15ms/step
934 [D loss: 8.20384

1/1 [==============================] - 0s 16ms/step
974 [D loss: 10.257971, acc: 55.00%] [G loss: 642978688.000000]
1/1 [==============================] - 0s 17ms/step
975 [D loss: 7.536879, acc: 55.00%] [G loss: 1963090304.000000]
1/1 [==============================] - 0s 17ms/step
976 [D loss: 11.518714, acc: 55.00%] [G loss: 1763306752.000000]
1/1 [==============================] - 0s 16ms/step
977 [D loss: 6.179247, acc: 50.00%] [G loss: 322729184.000000]
1/1 [==============================] - 0s 17ms/step
978 [D loss: 7.868384, acc: 60.00%] [G loss: 2448445952.000000]
1/1 [==============================] - 0s 16ms/step
979 [D loss: 8.146510, acc: 50.00%] [G loss: 992666304.000000]
1/1 [==============================] - 0s 16ms/step
980 [D loss: 9.006664, acc: 55.00%] [G loss: 3502078464.000000]
1/1 [==============================] - 0s 17ms/step
981 [D loss: 9.988230, acc: 50.00%] [G loss: 1434859648.000000]
1/1 [==============================] - 0s 18ms/step
982 [D loss: 8.848356